# Первый этап. Получаем массив из 300 сылок.

In [1]:
#подключаем необходимые библиотеки
import re
from urllib.parse import urlparse, parse_qsl, parse_qs
from lxml import html as lhtml
import json
from tqdm.notebook import tqdm
from multiprocessing.dummy import Pool as ThreadPool
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [13]:
tmp =''
top_games_list = []
for i in range(1,14):
    s = 'https://gg.deals/games/?sort=metascore&type=1&page=' + str(i)
    url = s
    tmp = requests.get(url).text
    soup2 = BeautifulSoup(tmp,'lxml')
    links2 = soup2.find_all('a', class_='details-line')
    links2 = [e['href'] for e in links2]
    top_games_list += links2

In [16]:
def href2(s):
    return 'https://gg.deals' + s

In [17]:
top_games_list = top_games_list[0:300]
top_games_list = list(map(href2,top_games_list))
top_games_list

['https://gg.deals/game/grand-theft-auto-v/',
 'https://gg.deals/game/half-life-2/',
 'https://gg.deals/game/half-life/',
 'https://gg.deals/game/bioshock/',
 'https://gg.deals/game/portal-2/',
 'https://gg.deals/game/divinity-original-sin-enhanced-edition/',
 'https://gg.deals/game/the-elder-scrolls-iv-oblivion-game-of-the-year-edition/',
 'https://gg.deals/game/mass-effect-2/',
 'https://gg.deals/game/quake/',
 'https://gg.deals/game/mass-effect-2-digital-deluxe-edition/',
 'https://gg.deals/game/the-elder-scrolls-v-skyrim/',
 'https://gg.deals/game/grand-theft-auto-vice-city/',
 'https://gg.deals/game/sid-meier-s-civilization-iv/',
 'https://gg.deals/game/bioshock-infinite/',
 'https://gg.deals/game/half-life-alyx/',
 'https://gg.deals/game/red-dead-redemption-ii/',
 'https://gg.deals/game/company-of-heroes-legacy-edition/',
 'https://gg.deals/game/hades/',
 'https://gg.deals/game/beat-saber/',
 'https://gg.deals/game/divinity-original-sin-2/',
 'https://gg.deals/game/minecraft/',
 

In [18]:
len(top_games_list)

300

# Второй этап.

In [277]:
def get_page(url, n_attempts):
    code_return = requests.get(url)
    while n_attempts != 0 :
        if code_return.status_code != 200:
            code_return = requests.get(url)
            n_attempts = n_attempts - 1
        else:
            break
    if code_return.status_code == 200:
        return code_return
    else:
        print('Error, failed request')
        print(code_return.status_code)
        return -1

In [278]:
import urllib ##function for take the url of the redirected url 


def resolve(url):
    return urllib.request.urlopen(url).geturl()

In [279]:
def take_counters(soup):
    cnt_mas = list()
    counter_soup = soup.find_all('div', class_=lambda s: s and s.endswith('game-action-wrap'))
    for i in counter_soup:
        cnt = i.find('span',class_ = 'count')
        if cnt is not None:
            cnt_mas.append(int(cnt.text))
    cnt_mas
    wish_cnt = cnt_mas[0]
    alert_cnt = cnt_mas[1]
    own_cnt = cnt_mas[2]
    return wish_cnt,alert_cnt,own_cnt

In [280]:
def take_reviews(soup):
    review_mas = list()
    check_review = soup.find('span',class_=lambda s: s and s.startswith('reviews-label'))
    if check_review is not None:
        try:
            review_positive_pctg = re.search(r'\d+%',str(check_review)).group(0)
        except BaseException:
            review_positive_pctg = -1
        reviews = check_review.text
        try:
            review_label = re.search(r'.+\(', reviews).group(0)[:-2]
        except BaseException:
            review_label = -1
        try:
            review_count = re.search(r'\d+.\d+',reviews).group(0)
        except BaseException:
            review_count = -1
    else:
        review_positive_pctg,review_label,review_count = [-1,-1,-1]
    return review_positive_pctg,review_label,review_count

In [281]:
def takes_info_release(soup):
    game_info_release = soup.find('div',class_='game-info-details-section game-info-details-section-release')
    game_info_developer = soup.find('div',class_ ='game-info-details-section game-info-details-section-developer')
    if game_info_release is not None:
        release_date = game_info_release.p.text
    else:
        release_date = -1
    if game_info_developer is not None:
        developer = game_info_developer.p.text
    else:
        developer = -1
    return release_date,developer

In [282]:
def takes_tags(soup):
    array_of_tags = list()
    tags_check = soup.find('div', id='game-info-tags')
    if tags_check:
        tags = tags_check.find_all('a')
        for i in tags:
            array_of_tags.append(i.text)
    return array_of_tags

In [283]:
def takes_features(soup):
    array_of_features = list()
    features_check = soup.find('div', id='game-info-features')
    if features_check:
        features = features_check.find_all('a')
        for i in features:
            array_of_features.append(i.text)
    return array_of_features

In [284]:
def takes_dlc_packs(id, arg):
    arr = list()
    headers = {'x-requested-with': 'XMLHttpRequest'}
    dlc_url = "https://gg.deals/ru/games/relations/{}/?type={}&ffset=0&hideKeyshops=0".format(id, arg)
    page = requests.get(dlc_url, headers=headers)
    if page.status_code == 200:
        tmp_soup = BeautifulSoup(page.text, 'lxml')
        tmp = tmp_soup.find_all('a', class_='game-info-title')
        for i in tmp:
            arr.append(i.text)
        return arr
    else:
        print("ERROR, Status:", page.status_code)
    return None

In [295]:
def takes_genres(soup):
    genre_tmp = soup.find('div', id='game-info-genres')
    genres = list()
    if genre_tmp is not None:
        genres_array = genre_tmp.find_all('a')
        for i in genres_array:
            genres.append(i.text)
        return genres
    return None

In [286]:
def get_id(soup):
    id = soup.find('div',class_=lambda s: s and s.startswith('wishlisted-game wishlisted-game') and s.endswith('game-action-wrap'))
    id_tmp = re.search(r'game-\d+\s', str(id)).group(0)
    id = id_tmp[5:-1]
    return id

In [287]:
def takes_pc_system(soup):
    system_array = []
    pc_system = soup.find_all('a', id = 'requirement-tab-trigger-pc_requirements')
    for e in pc_system:
        system_array.append(e.text)
    return system_array

In [288]:
def takes_meta_user(soup):
    sc = soup.find_all('span',class_=lambda s: s and s.startswith('progress-radial progress')) 
    if sc is not None:
        try:
            meta_score = sc[0].text
        except IndexError:
            meta_score = -1
        try:
            user_score = sc[1].text
        except IndexError:
            user_score = -1
    else:
        meta_score,user_score = -1
    return meta_score,user_score

In [289]:
def get_name(url):
    name = re.sub('https://gg.deals/game/','',url)
    name = name[0:-1]
    return name

In [290]:
def get_image(soup):
    image = soup.find('div', class_=lambda s: s and s.startswith('game-info-image'))
    image = image.find('img')
    image_url = image['src']
    return image_url

In [291]:
def get_market_url(soup):
    market = soup.find('div', class_=lambda s: s and s.startswith('game-info-image'))
    market = market.find('a')
    try:
        market_url = market['href']
        market_url = resolve(market_url)
    except TypeError:
        market_url = -1
    return market_url

In [292]:
def takes_price(id):
    url_for_graph = 'https://gg.deals/ru/games/chartHistoricalData/' + str(id) + '/?hideKeyshops=0'
    headers = {'x-requested-with': 'XMLHttpRequest' }
    graph_price = requests.get(url_for_graph, headers=headers)
    graph_json = graph_price.json()
    dict_price = {}
    price_history_list = []
    try:
        for i in range(0,len(graph_json['chartData']['deals'])):
            dict_price['ts'] = graph_json['chartData']['deals'][i]['x']
            dict_price['price'] = graph_json['chartData']['deals'][i]['y']
            dict_price['shop'] = graph_json['chartData']['deals'][i]['shop']
            price_history_list.append(dict_price)
            dict_price = {}
    except KeyError:
        price_history_list = []
    return price_history_list

In [296]:
def parse(url):
    name = get_name(url)
    code_return = get_page(url, n_attempts = 2)
    if code_return == -1:
            return -1
    inquiry = code_return.text
    soup_tmp = BeautifulSoup(inquiry,'lxml')
    image_url = get_image(soup_tmp)
    market_url = get_market_url(soup_tmp)
    wish_cnt,alert_cnt,own_cnt = take_counters(soup_tmp)
    review_positive_pctg,review_label,review_count = take_reviews(soup_tmp)
    release_date,developer = takes_info_release(soup_tmp)
    meta_score,user_score = takes_meta_user(soup_tmp)
    genres = takes_genres(soup_tmp)
    tags = takes_tags(soup_tmp)
    features = takes_features(soup_tmp)
    id_tmp = get_id(soup_tmp)
    dlcs_array = takes_dlc_packs(id_tmp, 'dlc')
    packs_href = takes_dlc_packs(id_tmp, 'packs')
    system_array = takes_pc_system(soup_tmp)
    price_history_list = takes_price(id_tmp)
    #############################################PACKS_ALL_INFO########################################################
    dict_game = {}
    dict_game['url'] = url
    dict_game['status'] = 'true'
    dict_game['name'] = name
    dict_game['image'] = image_url
    if release_date != -1:
        dict_game['release_date'] = release_date
    if developer != -1:
        dict_game['developer'] = developer
    if meta_score != -1:
        dict_game['metacritic_score'] = meta_score
    if user_score != -1:    
        dict_game['user_score'] = user_score
    if review_label != -1:
        dict_game['review_label'] = review_label
    if review_positive_pctg != -1:
        dict_game['review_positive_pctg'] = review_positive_pctg
    if review_count != -1:
        dict_game['review_count'] = review_count
    if genres is not None:
        dict_game['genres'] = genres
    if tags is not None:
        dict_game['tags'] = tags
    if features is not None:
        dict_game['features'] = features
    dict_game['pc_systems'] = system_array
    dict_game['wishlist_count'] = wish_cnt
    dict_game['alert_count'] = alert_cnt
    dict_game['owners_count'] = own_cnt
    if dlcs_array is not None:
        dict_game['dlcs'] = dlcs_array
    if packs_href is not None:
        dict_game['packs'] = packs_href
    if market_url != -1:
        dict_game['market_url'] = market_url
    if price_history_list != []:
        dict_game['price_history'] = price_history_list
    return dict_game

Использую многопоточность,записываю результаты в json файл.

In [301]:
import time
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool
with ThreadPool(50) as pool:
    try:
        r = list(tqdm(pool.imap(parse, top_games_list)))
    except BaseException:
        print('ERROR!')
        print(it)
    with open('D:/top_games_info.json', 'w') as f:
        json.dump(r, f)

300it [01:51,  2.69it/s]


In [300]:
r ### Тут r это вывод из 6 элементов первых, чтобы пример был в читаемом виде и не слишком длинный.

[{'url': 'https://gg.deals/game/grand-theft-auto-v/',
  'status': 'true',
  'name': 'grand-theft-auto-v',
  'image': 'https://img.gg.deals/79/83/9db226294b1e8632c5f235d18e49fa899bf2_307xt176.jpg',
  'release_date': '14 Apr 2015',
  'developer': 'Rockstar North',
  'metacritic_score': '96',
  'user_score': '7.8',
  'genres': ['Action', 'Adventure'],
  'tags': ['Action',
   'Adventure',
   'Atmospheric',
   'Automobile Sim',
   'Co-op',
   'Comedy',
   'Crime',
   'First-Person',
   'Funny',
   'Great Soundtrack',
   'Mature',
   'Moddable',
   'Multiplayer',
   'Open World',
   'Racing',
   'Sandbox',
   'Shooter',
   'Singleplayer',
   'Third Person',
   'Third-Person Shooter'],
  'features': ['Full controller support',
   'Remote Play on Phone',
   'Remote Play on Tablet',
   'Remote Play on TV',
   'Single-player',
   'Steam Achievements'],
  'pc_systems': ['Windows'],
  'wishlist_count': 3450,
  'alert_count': 663,
  'owners_count': 10136,
  'dlcs': ['Grand Theft Auto V - Criminal E